# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [1]:
import pandas as pd
import numpy as np
import math

from tqdm import tqdm

from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer

## Загрузка и предобработка данных

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
!wget -q -N https://www.dropbox.com/s/z8syrl5trawxs0n/articles.zip?dl=0 -O articles.zip
!unzip -o -q articles.zip

In [3]:
articles_df = pd.read_csv('articles/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [4]:
interactions_df = pd.read_csv('articles/users_interactions.csv')
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [5]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [6]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [7]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions:',len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [8]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [9]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [10]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [11]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [12]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(5)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


## Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [44]:
 #!pip install lightfm

In [14]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1. (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [15]:
interactions_full_df.shape, interactions_train_df.shape[0] + interactions_test_df.shape[0], interactions_train_df.shape, interactions_test_df.shape

((39106, 4), 39106, (29329, 4), (9777, 4))

In [16]:
pd.pivot_table(
    interactions_full_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)[pd.pivot_table(
    interactions_full_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0).columns[0]][0]

0.0

Поскольку трейн и тест должны быть одинаковы по размеру, то есть n пользователей х n статей, то сделаю трейн и тест одного размера, скопировав из оригинальной таблицы, а потом заполнив из трейна и теста взаимодействий

In [17]:
samp = pd.pivot_table(
    interactions_full_df,
    values='eventStrength',
    index='personId',
    columns='contentId').fillna(0)
print("Matrix from interactions was created!")
train_rate = samp.copy()
test_rate = samp.copy()
print("Train test equal matrix were copied from samp!")
for i in samp.columns:
  zer = np.zeros(samp[i].values.shape[0])
  train_rate[i].values[:] = zer
  test_rate[i].values[:] = zer
print("Train test were filled with 0 by their columns :) ")

Matrix from interactions was created!
Train test equal matrix were copied from samp!
Train test were filled with 0 by their columns :) 


Все встроенные методы немного ломались и не получалось использовать силу взаимодействия в них, поэтому делаю в лоб...

In [18]:
personIds = list(interactions_full_df.personId.unique()) 
for i in tqdm(range(len(personIds))):
  train_cop = interactions_train_df[interactions_train_df['personId'] == personIds[i]]
  test_cop = interactions_test_df[interactions_test_df['personId'] == personIds[i]]
  for j in range(len(train_cop.index)):
    train_rate.loc[personIds[i], train_cop['contentId']] = train_cop.iloc[j].eventStrength
  for z in range(len(test_cop.index)):
    test_rate.loc[personIds[i], test_cop['contentId']] = test_cop.iloc[z].eventStrength

100%|██████████| 1140/1140 [00:42<00:00, 27.09it/s]


In [19]:
sparse.csr_matrix(train_rate).shape

(1140, 2984)

In [20]:
# Ваш код здесь
data_train = sparse.csr_matrix(train_rate)
data_test = sparse.csr_matrix(test_rate)

## Задание 2. (1 балл)

Обучите модель LightFM с `loss='warp'` и посчитайте *precision@10* на тесте.

In [21]:
LF = LightFM(loss='warp', random_state=100)
LF.fit(data_train, epochs=10)
precision_at_k(model=LF, test_interactions=data_test, k=10).mean() # спасибо семинару за три строчки кода

0.0047861505

## Задание 3. (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss='warp'` и посчитайте precision@10 на тесте.

---

Значит нам нужно вытащить все тексты статей, которые встречаются в трейне, соединить их, векторизовать и добавить в модель, well

Копирую датасет, потому что потом я с ним сотворю нечто

In [22]:
df = articles_df.copy()

Делаю сортер и сортирую датасет со стьятьми в том порядке айдишек статей, в котором они лежат у меня в трейне

In [23]:
sorter = train_rate.columns

In [24]:
df.contentId = df.contentId.astype("category")
df.contentId.cat.set_categories(sorter, inplace=True)
df = df.loc[df['contentId'].isin(train_rate.columns)]
df = df.sort_values(["contentId"])

Создаю массив с текстами статей из датасета, он меньше необходимого, так как некоторые статьи были пропущены

In [25]:
tt = df['text'].to_numpy()

Нахожу потерянные статьи:

In [26]:
missing = list(set(train_rate.columns) - set(df['contentId'].tolist()))

Нахожу их индексы в трейне:

In [27]:
indices = []
for i in missing:
  indices.append(np.where(np.array(train_rate.columns) == i)[0][0])
indices = sorted(indices)

Вставляю в местах недостающих статей пустые строки и тем самым добиваю размер корпуса текстов до нужного

In [28]:
for i in indices:
  tt = np.insert(tt, i, '')

In [29]:
tt.shape

(2984,)

In [30]:
Tfidf = TfidfVectorizer(max_features=200)

In [31]:
# Ваш код здесь
feat = Tfidf.fit_transform(tt)

In [32]:
LF_tfidf = LightFM(loss='warp', random_state=10)
LF_tfidf.fit(data_train, item_features=feat, epochs=10)
precision_at_k(model=LF_tfidf, test_interactions=data_test, item_features=feat, k=10).mean() 

0.011710795

Ну, качество выросло хахах

## Задание 4. (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [90]:
# !pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from pymystem3 import Mystem

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [85]:
mystem = Mystem()
lemm = WordNetLemmatizer()
tokenizer_R = RegexpTokenizer(r'[\d.,]+|[A-Z][.A-Z]+\b\.*|\w+|\S')

In [35]:
Tfidf_preprocess = TfidfVectorizer(tokenizer=tokenizer_R.tokenize,
                                   max_features=300,
                                   # preprocessor=mystem.lemmatize,
                                   preprocessor=lemm.lemmatize,
                                   analyzer='word',
                                   stop_words=['english', 
                                               'spanish', 'portiguese'],
                                   smooth_idf=True,
                                   lowercase=True,
                                   min_df=3,
                                   max_df=2.0)
feat_preprocess = Tfidf_preprocess.fit_transform(tt)

In [36]:
LF_preprocess = LightFM(loss='warp', random_state=200)
LF_preprocess.fit(data_train, item_features=feat_preprocess, epochs=10)
precision_at_k(model=LF_preprocess, test_interactions=data_test, item_features=feat_preprocess, k=10).mean() 

0.010896131

Качество изменилось незначительно в худшую сторону, но, учитывая, насколько оно в целом плохое, это несильно что-то меняет. Видимо требуется более тяжелая артиллерия :(

Улучшилось ли качество предсказания?

## Задание 5. (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [37]:
# class lightfm.LightFM(no_components=10, k=5, n=10, learning_schedule=’adagrad’, loss=’logistic’, 
# learning_rate=0.05, rho=0.95, epsilon=1e-06, item_alpha=0.0, user_alpha=0.0, max_sampled=10, random_state=None)

In [38]:
no_components = [5, 10]
l_r = [0.01, 0.05, 0.07]
max_s = [5, 15]


In [39]:
from collections import defaultdict
d = defaultdict(lambda:defaultdict(lambda:0))

Да, это неоптимально, но для улучшения качества не потребовалось долго перебирать параметры, так что не так уж и неоптимально :3

In [40]:
for n_comp in tqdm(no_components):
  for learning_rate in l_r:
    for max_sampled in max_s:
      LF = LightFM(loss='warp', random_state=100, no_components=n_comp, learning_rate=learning_rate, max_sampled=max_sampled)
      LF.fit(data_train, item_features=feat_preprocess, epochs=20)
      prec = precision_at_k(model=LF, test_interactions=data_test, item_features=feat_preprocess, k=10).mean() 
      d[prec]['n_comp'] = n_comp
      d[prec]['l_r'] = learning_rate
      d[prec]['max_sampled'] = max_sampled

100%|██████████| 2/2 [01:48<00:00, 54.17s/it]


In [41]:
print(f"Лучшее получившееся качество было {max(d)}\nпри параметрах {sorted(d[max(d)].items())}")

Лучшее получившееся качество было 0.014460285194218159
при параметрах [('l_r', 0.07), ('max_sampled', 5), ('n_comp', 5)]


In [42]:
LF = LightFM(loss='warp', random_state=100, no_components=5, learning_rate=0.07, max_sampled=5)
LF.fit(data_train, item_features=feat_preprocess, epochs=20)
precision_at_k(model=LF, test_interactions=data_test, item_features=feat_preprocess, k=10).mean() 

0.014460285

## Бонусное задание. (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

---
Беру word2vec из прошлогодней домашки по текстам :)

In [182]:
import gensim
from gensim.models.word2vec import Word2Vec

In [172]:
# функции из той же домашки :3
def items_to_embedding(items: np.array, embedding_model):
    answer = [0.] * len(items)
    # for i in tqdm(range(len(items))):
    for i in range(len(items)):
        answer[i] = (sentence_embedding(items[i], embedding_model))
    return np.asarray(answer)

def sentence_embedding(sentence: str, embedding_model) -> np.array:
    """
    Складывает вектора токенов строки sentence
    """
    
    ans = np.zeros(embedding_model['system'].shape[0])
    for i in sentence:
        if i in embedding_model:
            emb = np.array(embedding_model[str(i)])
            ans += emb
    return ans / len(sentence)

Токенизируем тексты с заголовками, но сначала создадим такое в нашей таблице

In [153]:
articles_df['tittext'] = articles_df['title'] + ' ' + articles_df['text']
df = articles_df.copy()

# повторяю все действия только уже с объединнеными текстами
df.contentId = df.contentId.astype("category")
df.contentId.cat.set_categories(sorter, inplace=True)
df = df.loc[df['contentId'].isin(train_rate.columns)]
df = df.sort_values(["contentId"])

tittext = df['tittext'].to_numpy()

missing = list(set(train_rate.columns) - set(df['contentId'].tolist()))

indices = []
for i in missing:
  indices.append(np.where(np.array(train_rate.columns) == i)[0][0])
indices = sorted(indices)

for i in indices:
  tittext = np.insert(tittext, i, '')

tittext.shape[0] == 2984 # проверим размерность нового корпуса текстов

True

Теперь токенизируем все тексты

In [154]:
def tokenize(text):
  new_text = []
  tokens = tokenizer_R.tokenize(text)
  for i in tokens:
    token = lemm.lemmatize(i).lower()
    if token in stopwords.words(['english', 
                                 'spanish', 
                                 'portuguese']):
      pass
    else:
      new_text.append(token)
  return new_text

In [155]:
from tqdm import tqdm_notebook
new_tt = []
for i in tqdm_notebook(range(len(tittext) // 2984)):
  new_tt.append(tokenize(tittext[i]))

tt = pd.DataFrame(tittext, columns={'value' 
                                    #,'ind'
                                    })
tittext_new = tt['value'].apply(lambda x: tokenize(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [183]:
embedding_model = Word2Vec(sentences=tittext_new,
                           workers=3,
                           )

In [201]:
embedding = items_to_embedding(tittext_new, embedding_model)
embedding = np.nan_to_num(embedding)
feat = sparse.csc_matrix(embedding)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


Проверка на отсутствие пропусков и бесконечностей в данных, чтобы модель зафитилась спокойно!

In [198]:
np.any(np.isnan(embedding))

False

In [199]:
np.all(np.isfinite(embedding))

True

In [203]:
LF_FT = LightFM(loss='warp', random_state=50)
LF_FT.fit(data_train, item_features=feat, epochs=20)
precision_at_k(model=LF_FT, test_interactions=data_test, item_features=feat, k=10).mean()

0.010997962

Ну, качество не улучшилось, но эмбеддинг то мы обучили! :) Наверное, лучше было брать другой...